In [12]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.8 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np

df=pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [3]:
df.drop(columns=['BloodPressure','SkinThickness'],inplace=True,axis=1)
df.head()

,Pregnancies,Glucose,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,0,33.6,0.627,50,1
1,1,85,0,26.6,0.351,31,0
2,8,183,0,23.3,0.672,32,1
3,1,89,94,28.1,0.167,21,0
4,0,137,168,43.1,2.288,33,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   Insulin                   768 non-null    int64  
 3   BMI                       768 non-null    float64
 4   DiabetesPedigreeFunction  768 non-null    float64
 5   Age                       768 non-null    int64  
 6   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 42.1 KB


In [5]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scalar=StandardScaler()
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

X_train_scaled=scalar.fit_transform(X_train)
X_test_scaled=scalar.transform(X_test)

In [7]:
print(X_train.shape,y_train.shape)

(614, 6) (614,)


In [9]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

model=Sequential()

model.add(Dense(24,input_dim=6,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.fit(X_train_scaled,y_train,batch_size=24,validation_data=(X_test_scaled,y_test),epochs=100)

Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.5754 - loss: 0.6908 - val_accuracy: 0.6753 - val_loss: 0.6281
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6942 - loss: 0.6144 - val_accuracy: 0.7597 - val_loss: 0.5808
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7345 - loss: 0.5649 - val_accuracy: 0.7792 - val_loss: 0.5490
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7456 - loss: 0.5362 - val_accuracy: 0.7727 - val_loss: 0.5271
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7533 - loss: 0.5147 - val_accuracy: 0.7857 - val_loss: 0.5118
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7446 - loss: 0.5042 - val_accuracy: 0.7857 - val_loss: 0.5007
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7816 - loss: 0.4740 - val_accuracy: 0.7857 - val_loss: 0.4930
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7481 - loss: 0.5135 - val_accuracy: 0.7857 - 

In [13]:
import keras_tuner as kt

In [14]:
#for optimizer as hyperparameter
def build_model(hp):

  model=Sequential()
  model.add(Dense(24,activation='relu',input_dim=6))
  model.add(Dense(1,activation='sigmoid'))

  optimizers=hp.Choice('optimizers',values=['adam','rmsprop','adadelta','sgd'])

  model.compile(loss='binary_crossentropy',optimizer=optimizers,metrics=['accuracy'])

  return model

In [15]:
tuner=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      max_trials=5,
                      directory='mydir',
                      project_name='optimizer')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
tuner.search(X_train_scaled,y_train,epochs=5,validation_data=(X_test_scaled,y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 14s


In [17]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [18]:
tuner.get_best_hyperparameters()[0].values

{'optimizers': 'adam'}

In [19]:
model.fit(X_train_scaled,y_train,initial_epoch=5,epochs=100,validation_data=(X_test_scaled,y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.7466 - loss: 0.5674 - val_accuracy: 0.8117 - val_loss: 0.5321
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7628 - loss: 0.5405 - val_accuracy: 0.8182 - val_loss: 0.5125
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7596 - loss: 0.5146 - val_accuracy: 0.8052 - val_loss: 0.4982
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7568 - loss: 0.4873 - val_accuracy: 0.7922 - val_loss: 0.4878
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7619 - loss: 0.4950 - val_accuracy: 0.7922 - val_loss: 0.4825
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7634 - loss: 0.4988 - val_accuracy: 0.7987 - val_loss: 0.4776
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7389 - loss: 0.5147 - val_accuracy: 0.7987 - val_loss: 0.4738
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7558 - loss: 0.4855 - val_accuracy: 0.792

In [25]:
def build_model(hp):
  model=Sequential()

  units=hp.Int('nodes',min_value=6,max_value=132)

  model.add(Dense(units=units,activation='relu',input_dim=6))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [26]:
tuner=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      max_trials=5,
                      directory='mydir',
                      project_name='nodes'
                      )

Reloading Tuner from mydir/nodes/tuner0.json


In [32]:
tuner.get_best_hyperparameters()[0].values

{'nodes': 48}

In [27]:
tuner.search(X_train_scaled,y_train,validation_data=(X_test_scaled,y_test),epochs=5)

In [31]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [33]:
model.fit(X_train_scaled,y_train,initial_epoch=5,epochs=100,validation_data=(X_test_scaled,y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.7833 - loss: 0.4753 - val_accuracy: 0.7922 - val_loss: 0.4963
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7570 - loss: 0.4976 - val_accuracy: 0.7922 - val_loss: 0.4901
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7783 - loss: 0.4751 - val_accuracy: 0.7987 - val_loss: 0.4859
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7768 - loss: 0.4601 - val_accuracy: 0.7922 - val_loss: 0.4821
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7411 - loss: 0.4937 - val_accuracy: 0.7857 - val_loss: 0.4801
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7809 - loss: 0.4569 - val_accuracy: 0.7857 - val_loss: 0.4770
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7535 - loss: 0.4936 - val_accuracy: 0.7792 - val_loss: 0.4755
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7638 - loss: 0.4599 - val_accuracy: 0.779

COMPACT

In [118]:
def build_model(hp):

  model=Sequential()
  counter=0

  for i in range(hp.Int('layers',min_value=1,max_value=10)):

    if counter==0:

      model.add(Dense(
                units=hp.Int('units' + str(i),min_value=6,max_value=132,step=6),
                activation=hp.Choice('activation' + str(i),values=['relu','sigmoid','tanh']),
                input_dim=6
                )
            )

      model.add(Dropout(hp.Choice('dropout' + str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    else:
      model.add(Dense(
                units=hp.Int('units' + str(i),min_value=6,max_value=132,step=6),
                activation=hp.Choice('activation' + str(i),values=['relu','sigmoid','tanh'])
           )
      )

      model.add(Dropout(hp.Choice('dropout' + str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    counter+=1

  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizers',values=['adam','sgd','rmsprop','nadam','adadelta']),
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

In [119]:
tuner1=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      max_trials=3,
                      directory='mydir',
                      project_name='Pakka_final_hai')

Reloading Tuner from mydir/Pakka_final_hai/tuner0.json


In [120]:
tuner1.search(X_train_scaled,y_train,validation_data=(X_test_scaled,y_test),epochs=5)

In [121]:
tuner1.get_best_hyperparameters()[0].values

{'layers': 5,
 'units0': 48,
 'activation0': 'relu',
 'dropout0': 0.4,
 'optimizers': 'rmsprop',
 'units1': 6,
 'activation1': 'relu',
 'dropout1': 0.6,
 'units2': 48,
 'activation2': 'relu',
 'dropout2': 0.6,
 'units3': 102,
 'activation3': 'tanh',
 'dropout3': 0.1,
 'units4': 132,
 'activation4': 'relu',
 'dropout4': 0.2,
 'units5': 18,
 'activation5': 'relu',
 'dropout5': 0.2,
 'units6': 78,
 'activation6': 'tanh',
 'dropout6': 0.6,
 'units7': 12,
 'activation7': 'tanh',
 'dropout7': 0.1}

In [122]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [123]:
model.fit(X_train_scaled,y_train,initial_epoch=5,epochs=200,batch_size=24,validation_data=(X_test_scaled,y_test))

Epoch 6/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.6672 - loss: 0.6636 - val_accuracy: 0.7143 - val_loss: 0.5361
Epoch 7/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6565 - loss: 0.6546 - val_accuracy: 0.7143 - val_loss: 0.5300
Epoch 8/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6946 - loss: 0.5960 - val_accuracy: 0.7403 - val_loss: 0.5177
Epoch 9/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6584 - loss: 0.6535 - val_accuracy: 0.7597 - val_loss: 0.5122
Epoch 10/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7100 - loss: 0.6005 - val_accuracy: 0.7532 - val_loss: 0.5097
Epoch 11/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6632 - loss: 0.6631 - val_accuracy: 0.7532 - val_loss: 0.5076
Epoch 12/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6781 - loss: 0.6441 - val_accuracy: 0.7468 - val_loss: 0.5057
Epoch 13/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6593 - loss: 0.6431 - val_accuracy: 0.759